In [6]:
import json

# تحميل البيانات من ملف JSON
with open(r"D:\Desktop\Programming\LangChain - learning\ice_breaker\my_projects\agentsqlprojectbot\token_logs\chat_5283091780_costs.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# عدد الطلبات الحالية حسب الملف
current_requests = 999  # حسب ما ذكرت
future_requests = 10**6

total_cost_all_models = 0
estimated_total_cost_all_models = 0

for model in data.get("models", []):
    # تحديث سعر gemini-2.5-flash
    if model["name"] == "gemini-2.5-flash":
        model["pricing"]["output_per_1m"] = 2.5
        model["pricing"]["input_per_1m"] = 0.3

    # التكلفة الحالية
    input_cost = (model["tokens"]["input"] / 1_000_000) * model["pricing"]["input_per_1m"]
    output_cost = (model["tokens"]["output"] / 1_000_000) * model["pricing"]["output_per_1m"]
    total_cost = input_cost + output_cost
    total_cost_all_models += total_cost

    # التكلفة المستقبلية (10,000 طلب)
    scale_factor = future_requests / current_requests
    estimated_total_cost = total_cost * scale_factor
    estimated_total_cost_all_models += estimated_total_cost

    print(f"Model: {model['name']}")
    print(f"  Current Total Cost: ${total_cost:.6f}")
    print(f"  Estimated Total Cost for {future_requests} requests: ${estimated_total_cost:.2f}")
    print("-" * 40)

print(f"Total Current Cost (all models): ${total_cost_all_models:.6f}")
print(f"Estimated Total Cost for {future_requests} requests (all models): ${estimated_total_cost_all_models:.2f}")


Model: gemini-2.5-flash
  Current Total Cost: $0.074689
  Estimated Total Cost for 1000000 requests: $74.76
----------------------------------------
Model: gemini-2.0-flash
  Current Total Cost: $0.003852
  Estimated Total Cost for 1000000 requests: $3.86
----------------------------------------
Total Current Cost (all models): $0.078541
Estimated Total Cost for 1000000 requests (all models): $78.62


In [1]:
from sqlalchemy import create_engine, MetaData, inspect

# رابط الاتصال
# connection_string = "mssql+pyodbc://@B515R\\SQLEXPRESS/manager?driver=ODBC+Driver+17+for+SQL+Server"
connection_string = "sqlite:///example.db"

# إنشاء محرك SQLAlchemy
engine = create_engine(connection_string)

# إنشاء كائن Metadata
metadata = MetaData()

# عمل Reflect لجميع الجداول
metadata.reflect(bind=engine)

# استخدام Inspector للحصول على معلومات أكثر تفصيلًا
inspector = inspect(engine)

print("=== Tables in database ===")
for table_name in inspector.get_table_names():
    print(f"\nTable: {table_name}")
    
    # أعمدة الجدول
    columns = inspector.get_columns(table_name)
    for col in columns:
        print(f"  {col['name']} ({col['type']}) Nullable:{col['nullable']} Default:{col['default']}")

    # المفاتيح الأساسية
    pk = inspector.get_pk_constraint(table_name)
    print(f"  Primary Key: {pk.get('constrained_columns')}")

    # المفاتيح الأجنبية
    fks = inspector.get_foreign_keys(table_name)
    for fk in fks:
        print(f"  Foreign Key: {fk.get('constrained_columns')} -> {fk.get('referred_table')}({fk.get('referred_columns')})")


=== Tables in database ===


In [2]:
from sqlalchemy import create_engine, text, inspect


In [6]:
from sqlalchemy import create_engine, text, inspect

async def get_examples_from_connection_string(self, connection_string: str):
    """✅ الحصول على سكيما الجداول وأول صف من كل جدول - محسّن مع دعم SQLite"""
    try:
        print(f"📌 رابط الاتصال: {connection_string}")
        
        # ✅ إنشاء محرك بشكل مباشر (بدون executor)
        from sqlalchemy import create_engine, MetaData, text
        
        print("🔄 جاري إنشاء محرك قاعدة البيانات...")
        engine = create_engine(connection_string)
        
        # ✅ اختبار الاتصال أولاً
        try:
            with engine.connect() as conn:
                print("✅ تم الاتصال بقاعدة البيانات بنجاح")
        except Exception as e:
            print(f"❌ فشل الاتصال: {e}")
            return None, None, "db_type"
        
        # ✅ استخدام Inspector للحصول على الجداول
        print("🔍 جاري البحث عن الجداول...")
        inspector = inspect(engine)
        table_names = inspector.get_table_names()
        
        print(f"📋 الجداول المكتشفة: {table_names}")
        print(f"📊 العدد الكلي: {len(table_names)}")
        
        if not table_names:
            print("⚠️ لم يتم العثور على جداول قابلة للاستخدام")
            return None, None, "db_type"
        
        print(f"✅ وجد {len(table_names)} جدول صالح للاستخدام\n")
        
        schema_parts = []
        data_parts = []
        
        # ✅ معالجة كل جدول
        for idx, table_name in enumerate(table_names, 1):
            print(f"\n{'='*60}")
            print(f"[{idx}/{len(table_names)}] 🔨 معالجة الجدول: {table_name}")
            print(f"{'='*60}")
            
            # تجاهل جداول النظام
            if table_name.lower() in ['sysdiagrams', 'pg_stat_statements', 'sqlite_sequence']:
                print(f"⏭️ تم تجاهل جدول النظام: {table_name}")
                continue
            
            try:
                # 1️⃣ الحصول على أسماء الأعمدة والأنواع
                columns = inspector.get_columns(table_name)
                if not columns:
                    print(f"⚠️ لا توجد أعمدة في الجدول {table_name}")
                    continue
                
                print(f"📍 عدد الأعمدة: {len(columns)}")
                
                column_info = []
                for col in columns:
                    col_name = col['name']
                    col_type = str(col['type'])
                    nullable = "NULL" if col['nullable'] else "NOT NULL"
                    col_default = col.get('default', None)
                    default_str = f" DEFAULT {col_default}" if col_default else ""
                    column_info.append(f"{col_name} ({col_type}) {nullable}{default_str}")
                
                # 2️⃣ الحصول على المفاتيح الأساسية
                pk = inspector.get_pk_constraint(table_name)
                pk_columns = pk.get('constrained_columns', []) if pk else []
                print(f"🔑 المفتاح الأساسي: {pk_columns if pk_columns else 'لا يوجد'}")
                
                # 3️⃣ الحصول على العلاقات (Foreign Keys)
                foreign_keys = inspector.get_foreign_keys(table_name)
                fk_info = []
                for fk in foreign_keys:
                    fk_columns = ', '.join(fk['constrained_columns'])
                    ref_table = fk['referred_table']
                    ref_columns = ', '.join(fk['referred_columns'])
                    fk_info.append(f"{fk_columns} -> {ref_table}({ref_columns})")
                
                print(f"🔗 العلاقات: {len(fk_info) if fk_info else 'لا توجد'}")
                
                # 4️⃣ بناء السكيما المفصلة
                schema_line = f"\n📊 جدول: {table_name}\n"
                schema_line += f"  الأعمدة ({len(columns)}):\n"
                
                for col_info_item in column_info:
                    col_name = col_info_item.split()[0]
                    if col_name in pk_columns:
                        schema_line += f"    🔑 {col_info_item}\n"
                    else:
                        schema_line += f"    • {col_info_item}\n"
                
                if fk_info:
                    schema_line += f"\n  العلاقات (Foreign Keys):\n"
                    for fk_item in fk_info:
                        schema_line += f"    🔗 {fk_item}\n"
                
                schema_parts.append(schema_line)
                print(f"✅ تم استخراج السكيما بنجاح")
                
            except Exception as e:
                print(f"⚠️ خطأ في معالجة الجدول {table_name}: {e}")
                import traceback
                traceback.print_exc()
                continue
            
            # 5️⃣ الحصول على أول صف (مثال)
            try:
                sample_query = f"SELECT * FROM \"{table_name}\" LIMIT 1"
                print(f"🔍 تنفيذ الاستعلام: {sample_query}")
                
                # تنفيذ الاستعلام بشكل مباشر
                with engine.connect() as connection:
                    result = connection.execute(text(sample_query))
                    row = result.fetchone()
                    
                    if row:
                        # تحويل الصف إلى صيغة مقروءة
                        col_names = [col['name'] for col in columns]
                        row_dict = dict(zip(col_names, row))
                        
                        # طباعة جميلة للبيانات
                        row_str = " | ".join(f"{k}: {v}" for k, v in row_dict.items())
                        data_parts.append(f"\n📋 مثال من {table_name}:\n  {row_str}\n")
                        print(f"✅ تم الحصول على بيانات العينة:")
                        print(f"   {row_str}")
                    else:
                        print(f"⚠️ الجدول {table_name} فارغ (لا توجد بيانات)")
                        data_parts.append(f"\n📋 جدول {table_name}: (فارغ - لا توجد بيانات)\n")
                        
            except Exception as e:
                print(f"⚠️ خطأ في جلب مثال من {table_name}: {e}")
                import traceback
                traceback.print_exc()
        
        # ✅ تجميع النتائج النهائية
        schema_text = "\n".join(schema_parts) if schema_parts else "لا توجد بيانات سكيما متاحة"
        data_text = "\n".join(data_parts) if data_parts else "لا توجد بيانات عينة متاحة"
        
        print(f"\n{'='*60}")
        print(f"✅ اكتمل استخراج البيانات بنجاح!")
        print(f"{'='*60}")
        print(f"📊 ملخص النتائج:")
        print(f"   • عدد الجداول المعالجة: {len(schema_parts)}")
        print(f"   • حجم السكيما: ~{len(schema_text)} حرف")
        print(f"   • حجم البيانات: ~{len(data_text)} حرف")
        
        return schema_text, data_text, "db_type"

    except Exception as e:
        print(f"❌ خطأ في الحصول على الأمثلة: {e}")
        import traceback
        traceback.print_exc()
        return None, None, 'unknown'
    
x = await get_examples_from_connection_string(None, r"sqlite:///D:\Desktop\Programming\LangChain - learning\ice_breaker\my_projects\agentsqlprojectbot\services\example.db")
print(x)

📌 رابط الاتصال: sqlite:///D:\Desktop\Programming\LangChain - learning\ice_breaker\my_projects\agentsqlprojectbot\services\example.db
🔄 جاري إنشاء محرك قاعدة البيانات...
✅ تم الاتصال بقاعدة البيانات بنجاح
🔍 جاري البحث عن الجداول...
📋 الجداول المكتشفة: ['Orders', 'Users']
📊 العدد الكلي: 2
✅ وجد 2 جدول صالح للاستخدام


[1/2] 🔨 معالجة الجدول: Orders
📍 عدد الأعمدة: 5
🔑 المفتاح الأساسي: ['order_id']
🔗 العلاقات: 1
✅ تم استخراج السكيما بنجاح
🔍 تنفيذ الاستعلام: SELECT * FROM "Orders" LIMIT 1
✅ تم الحصول على بيانات العينة:
   order_id: 1 | user_id: 2 | product_name: Anything | amount: 179.92 | order_date: 2025-07-30

[2/2] 🔨 معالجة الجدول: Users
📍 عدد الأعمدة: 5
🔑 المفتاح الأساسي: ['user_id']
🔗 العلاقات: لا توجد
✅ تم استخراج السكيما بنجاح
🔍 تنفيذ الاستعلام: SELECT * FROM "Users" LIMIT 1
✅ تم الحصول على بيانات العينة:
   user_id: 1 | name: Laura Rogers | email: jacksonjames@example.com | city: Webbfurt | age: 33

✅ اكتمل استخراج البيانات بنجاح!
📊 ملخص النتائج:
   • عدد الجداول المعالجة: 2
   • حج

In [6]:
import os
print(os.path.abspath("example.db"))


d:\Desktop\Programming\LangChain - learning\ice_breaker\my_projects\agentsqlprojectbot\utils\example.db


In [ ]:
import hashlib

import secrets

password = "beko515"
salt = secrets.token_hex(16)
hashed = hashlib.pbkdf2_hmac('sha256', password.encode(), salt.encode(), 100000)
print (f"{salt}${hashed.hex()}")

d7de38d2b6d5162eb62a5770adadaa7d$060aed066c2485bf07bd4212e703d8f4a71e12277d7affdbc1201314aa043a6b


In [ ]:
import sqlite3
from faker import Faker
import random

# إنشاء كائن Faker لتوليد بيانات وهمية
fake = Faker()

# الاتصال بقاعدة البيانات (تُنشأ تلقائيًا إن لم تكن موجودة)
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

# إنشاء الجداول
cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT,
    city TEXT,
    age INTEGER
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    product_name TEXT,
    amount REAL,
    order_date TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)
""")

# إدخال بيانات وهمية في جدول Users
for _ in range(10):
    name = fake.name()
    email = fake.email()
    city = fake.city()
    age = random.randint(18, 60)
    cursor.execute("INSERT INTO Users (name, email, city, age) VALUES (?, ?, ?, ?)",
                   (name, email, city, age))

# إدخال بيانات وهمية في جدول Orders
for _ in range(10):
    user_id = random.randint(1, 10)
    product_name = fake.word().capitalize()
    amount = round(random.uniform(10, 500), 2)
    order_date = fake.date_this_year().isoformat()
    cursor.execute("INSERT INTO Orders (user_id, product_name, amount, order_date) VALUES (?, ?, ?, ?)",
                   (user_id, product_name, amount, order_date))

# حفظ التغييرات وإغلاق الاتصال
conn.commit()
conn.close()

print("✅ SQLite database created successfully with fake data (example.db)")


📋 Tables in database:
- Users
- Orders
